In [1]:
pip install sec_edgar_downloader

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import re
from bs4 import BeautifulSoup
from sec_edgar_downloader import Downloader

# ✅ MAIN FOLDER STRUCTURE CONFIG
DATA_DIR = "sec-edgar-filings"

# ✅ 1. Download latest filings into your target folder
def download_latest_filing(ticker: str = "NVDA", form_type: str = "10-Q") -> str:
    # Ensure the base directory exists
    os.makedirs(DATA_DIR, exist_ok=True)

    dl = Downloader("dvschenone@ucsd.edu", DATA_DIR)
    dl.get(form_type, ticker)

    return get_latest_filing_file(ticker, form_type)




# ✅ 2. Traverse folders to find latest primary-document
from datetime import datetime

def get_latest_filing_file(ticker: str, form_type: str, year_threshold: int = 2020) -> str:
    filings_path = os.path.join(DATA_DIR, ticker, form_type)
    latest_file = None
    latest_mtime = 0

    if not os.path.exists(filings_path):
        print(f"❌ Path does not exist: {filings_path}")
        return None

    for root, _, files in os.walk(filings_path):
        # Extract year from folder name (e.g., 0001012870-22-000123)
        folder_name = os.path.basename(root)
        year_match = re.search(r'-(\d{2})-', folder_name)
        if year_match:
            year = int("20" + year_match.group(1))  # Convert e.g. "22" → 2022
            if year < year_threshold:
                continue  # skip old filings

        for file in files:
            if file.endswith(".html") or file.endswith(".txt"):
                file_path = os.path.join(root, file)
                mtime = os.path.getmtime(file_path)

                if mtime > latest_mtime:
                    latest_file = file_path
                    latest_mtime = mtime

    return latest_file






# ✅ 3. Extract "Item 1A. Risk Factors" from .html or .txt
def extract_item_1a(file_path: str) -> str:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        content = f.read()

    # If it's HTML, parse and extract just the text
    if "<html" in content.lower():
        soup = BeautifulSoup(content, "html.parser")
        text = soup.get_text(separator="\n")
    else:
        text = content

    # Normalize and extract Item 1A section
    text = re.sub(r'\n+', '\n', text)
    pattern = r'(?i)(item\s+1A\.?\s*[-–:]?\s*Risk\s+Factors)(.*?)(item\s+1B\.?|item\s+2\.)'
    match = re.search(pattern, text, re.DOTALL)

    if match:
        return match.group(2).strip()
    else:
        return "Item 1A section not found."

# ✅ 4. Example run
if __name__ == "__main__":
    file_path = get_latest_filing_file("NVDA", "10-Q", year_threshold=2020)

    if file_path:
        print(f"✅ Found latest filing: {file_path}")
        print(extract_item_1a(file_path)[:1000])
    else:
        print("❌ Still no filing found.")





🔍 Looking in: sec-edgar-filings/NVDA/10-Q
📄 Found file: full-submission.txt
📄 Found file: full-submission.txt
📄 Found file: full-submission.txt
📄 Found file: full-submission.txt
📄 Found file: full-submission.txt
📄 Found file: full-submission.txt
📄 Found file: full-submission.txt
📄 Found file: full-submission.txt
📄 Found file: full-submission.txt
📄 Found file: full-submission.txt
📄 Found file: full-submission.txt
📄 Found file: full-submission.txt
📄 Found file: full-submission.txt
📄 Found file: full-submission.txt
📄 Found file: full-submission.txt
📄 Found file: full-submission.txt
📄 Found file: full-submission.txt
📄 Found file: full-submission.txt
📄 Found file: full-submission.txt
📄 Found file: full-submission.txt
📄 Found file: full-submission.txt
📄 Found file: full-submission.txt
📄 Found file: full-submission.txt
📄 Found file: full-submission.txt
📄 Found file: full-submission.txt
📄 Found file: full-submission.txt
📄 Found file: full-submission.txt
📄 Found file: full-submission.txt
📄 Foun